<a href="https://colab.research.google.com/github/cjsdudwls1/simple_things/blob/main/0908_%EC%95%84%ED%94%8C%EB%9D%BC%ED%86%A1%EC%8B%A0_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 특성변수 생성
2. 결측치는 전날데이터 채우기
3. 변수는 말씀하신 3개만
4. 로지스틱 class_wight=balanced 활용하여 회귀
5. 파이캐럿으로 스모트해서 넣기
6. 샤프 활용

In [ ]:
!pip install tensorflow -q

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# --- 1단계: 데이터 로드 ---
X_df = pd.read_pickle('/content/drive/MyDrive/식약처(아플라톡신 관련 데이터)/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신/_학습용 데이터_/df_통합_LIMS_기상정보_결합_gzip_X.pkl', compression='gzip')
y_df = pd.read_pickle('/content/drive/MyDrive/식약처(아플라톡신 관련 데이터)/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신/_학습용 데이터_/df_통합_LIMS_기상정보_결합_gzip_y.pkl', compression='gzip')

In [ ]:
X_df

,INSPCT_PURPS_NAME,INSPCT_KND_NAME,PRDLST_NM,SPLORE_STTUS_NAME,ORGPLCE_NATION_CODE,tmprt_150_29,tmprt_150Top_29,tmprt_150Lwet_29,hd_150_29,arvlty_300_29,...,tmprt_150_00,tmprt_150Top_00,tmprt_150Lwet_00,hd_150_00,arvlty_300_00,arvlty_300Top_00,afp_00,sunshn_Time_00,solrad_Qy_00,soil_Mitr_10_00
0,자가품질위탁검사,NaN,과자,냉동,NaN,-5.4,1.3,-10.6,59.3,0.3,...,-4.8,1.9,-9.9,77.9,0.2,2.8,1.0,NaN,3.99,16.1
1,자가품질위탁검사,NaN,과자,냉동,NaN,-5.4,1.3,-10.6,59.3,0.3,...,-4.8,1.9,-9.9,77.9,0.2,2.8,1.0,NaN,3.99,16.1
2,자가품질위탁검사,식품,보스웰리아추출물등 복합물(Flexir)(제2021-9호),실온,NaN,-2.6,3.0,-7.6,55.9,0.7,...,-2.9,5.7,-10.2,79.6,0.6,6.0,0.5,NaN,6.11,NaN
3,자가품질위탁검사,식품,보스웰리아추출물등 복합물(Flexir)(제2021-9호),실온,NaN,-2.6,3.0,-7.6,55.9,0.7,...,-2.9,5.7,-10.2,79.6,0.6,6.0,0.5,NaN,6.11,NaN
4,자가품질위탁검사,식품,과자,NaN,NaN,-1.8,2.7,-7.4,46.9,0.5,...,-2.5,4.4,-8.2,77.2,0.4,4.2,0.0,92.0,4.73,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,모니터링검사,식품,옥수수건조,NaN,NaN,-2.0,4.3,-9.2,65.3,2.5,...,1.8,3.5,-0.2,86.0,2.3,9.5,11.0,NaN,4.18,16.3
170742,모니터링검사,식품,옥수수건조,NaN,NaN,-2.0,4.3,-9.2,65.3,2.5,...,1.8,3.5,-0.2,86.0,2.3,9.5,11.0,NaN,4.18,16.3
170743,모니터링검사,식품,옥수수건조,NaN,NaN,-2.0,4.3,-9.2,65.3,2.5,...,1.8,3.5,-0.2,86.0,2.3,9.5,11.0,NaN,4.18,16.3
170744,모니터링검사,식품,옥수수건조,NaN,NaN,-2.0,4.3,-9.2,65.3,2.5,...,1.8,3.5,-0.2,86.0,2.3,9.5,11.0,NaN,4.18,16.3


# JDGMNT_WORD_NAME 컬럼에서 "상기실험확인함" 값는 갖는 데이터 전부 삭제

In [ ]:
# Filter out rows where 'JDGMNT_WORD_NAME' is '상기실험확인함'
filtered_indices = y_df[y_df['JDGMNT_WORD_NAME'] != '상기실험확인함'].index

X_df_filtered = X_df.loc[filtered_indices]
y_df_filtered = y_df.loc[filtered_indices]

print("Original X_df shape:", X_df.shape)
print("Filtered X_df shape:", X_df_filtered.shape)
print("Original y_df shape:", y_df.shape)
print("Filtered y_df shape:", y_df_filtered.shape)

Original X_df shape: (170746, 305)
Filtered X_df shape: (130657, 305)
Original y_df shape: (170746, 3)
Filtered y_df shape: (130657, 3)


In [ ]:
y_df_filtered['JDGMNT_WORD_NAME'].value_counts(normalize=True)

,proportion
JDGMNT_WORD_NAME,
적합,0.995699
부적합,0.004301


In [ ]:
# Calculate missing values count and percentage
missing_data = X_df.isnull().sum()
missing_percent = 100 * missing_data / len(X_df)

# Get sample non-missing values for each column
def get_sample_values(series):
    non_null_values = series.dropna().unique()
    if len(non_null_values) > 5:
        return non_null_values[:5].tolist()
    else:
        return non_null_values.tolist()

sample_values = X_df.apply(get_sample_values)

# Create a DataFrame to summarize the information
missing_info_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percent (%)': missing_percent,
    'Sample Values': sample_values
})

# Display the DataFrame
display(missing_info_df)

,Missing Count,Missing Percent (%),Sample Values
INSPCT_PURPS_NAME,0,0.000000,"[자가품질위탁검사, 수거검사, 품질검사, 기타, 식품안전관리인증검사]"
INSPCT_KND_NAME,38481,22.536985,"[식품, 축산물, 농산물, 수산물]"
PRDLST_NM,0,0.000000,"[과자, 보스웰리아추출물등 복합물(Flexir)(제2021-9호), 땅콩 또는 견과..."
SPLORE_STTUS_NAME,86845,50.862099,"[냉동, 실온, 냉장, 완제품]"
ORGPLCE_NATION_CODE,168345,98.593818,"[국산, 대한민국, 베트남, 한국, 중국]"
...,...,...,...
arvlty_300Top_00,17369,10.172420,"[2.8, 6.0, 4.2, 0.0, 7.9]"
afp_00,52,0.030455,"[1.0, 0.5, 0.0, 2.0, 7.0]"
sunshn_Time_00,137443,80.495590,"[92.0, 17.0, 206.0, 466.0, 221.0]"
solrad_Qy_00,38783,22.713856,"[3.99, 6.11, 4.73, 9.52, 4.25]"


In [ ]:
# Filter missing_info_df to show columns with missing percent <= 30%
filtered_missing_info_df = missing_info_df[missing_info_df['Missing Percent (%)'] <= 20]

# Display the filtered DataFrame
display(filtered_missing_info_df)

,Missing Count,Missing Percent (%),Sample Values
INSPCT_PURPS_NAME,0,0.000000,"[자가품질위탁검사, 수거검사, 품질검사, 기타, 식품안전관리인증검사]"
PRDLST_NM,0,0.000000,"[과자, 보스웰리아추출물등 복합물(Flexir)(제2021-9호), 땅콩 또는 견과..."
tmprt_150_29,46,0.026941,"[-5.4, -2.6, -1.8, -5.5, 0.2]"
tmprt_150Top_29,46,0.026941,"[1.3, 3.0, 2.7, 1.8, 7.6]"
tmprt_150Lwet_29,46,0.026941,"[-10.6, -7.6, -7.4, -11.6, -5.7]"
...,...,...,...
tmprt_150Lwet_00,51,0.029869,"[-9.9, -10.2, -8.2, -13.1, -7.2]"
hd_150_00,108,0.063252,"[77.9, 79.6, 77.2, 79.9, 81.2]"
arvlty_300_00,17368,10.171834,"[0.2, 0.6, 0.4, 0.0, 0.3]"
arvlty_300Top_00,17369,10.172420,"[2.8, 6.0, 4.2, 0.0, 7.9]"


In [ ]:
y_df_filtered

,JDGMNT_WORD_NAME,결과,결과값
0,적합,불검출,0
1,적합,불검출,0
2,적합,불검출,0
3,적합,불검출,0
4,적합,불검출,0
...,...,...,...
170665,적합,불검출,0
170668,적합,불검출,0
170669,적합,불검출,0
170670,적합,불검출,0


# 데이터를 학습 세트와 테스트 세트로 분할합니다. 'is_zero' 컬럼을 기준으로 층화 샘플링을 수행하여 클래스 비율을 유지하고, 분할된 세트의 크기를 출력합니다.

In [ ]:
# # 6. 데이터를 학습 세트와 테스트 세트로 분할합니다. X_df_filtered와 y_df_filtered를 사용하며, 'is_zero' 컬럼을 층화 추출의 기준으로 활용하여 클래스 비율이 유지되도록 분할합니다. 분할 비율은 8:2로 설정합니다.
# X_train, X_test, y_train, y_test = train_test_split(
#     X_df_filtered,
#     y_df_filtered,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_df_filtered['JDGMNT_WORD_NAME'] # Stratify based on the 'is_zero' column
# )

# # 7. 분할된 학습 세트와 테스트 세트의 크기를 출력하여 데이터 분할이 올바르게 이루어졌는지 확인합니다.
# print("\nShape of X_train:", X_train.shape)
# print("Shape of X_test:", X_test.shape)
# print("Shape of y_train:", y_train.shape)
# print("Shape of y_test:", y_test.shape)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import numpy as np
from scipy.sparse import hstack

# 1. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_df_filtered,
    y_df_filtered,
    test_size=0.2,
    random_state=42,
    stratify=y_df_filtered['JDGMNT_WORD_NAME']
)

# 2. 타겟 변수 인코딩
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train['JDGMNT_WORD_NAME'])
y_test_encoded = le.transform(y_test['JDGMNT_WORD_NAME'])
print("Class mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# 3. 범주형 피처 인코딩 (희소행렬로 변환) - Use original X_train for fitting
object_cols = X_train.select_dtypes(include='object').columns
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

X_train_encoded_ohe = ohe.fit_transform(X_train[object_cols]) # Fit on original X_train
X_test_encoded_ohe = ohe.transform(X_test[object_cols]) # Transform X_test

# 수치형 피처는 그대로 유지
numeric_cols = X_train.drop(columns=object_cols).columns
X_train_numeric = X_train[numeric_cols].values
X_test_numeric = X_test[numeric_cols].values

# 범주형 + 수치형 합치기 (희소행렬)
X_train_all = hstack([X_train_encoded_ohe, X_train_numeric])
X_test_all = hstack([X_test_encoded_ohe, X_test_numeric])


# 4. 결측치 처리 (SimpleImputer → 빠름) - Impute combined data
imputer = SimpleImputer(strategy="most_frequent")
X_train_imputed = imputer.fit_transform(X_train_all.toarray())
X_test_imputed = imputer.transform(X_test_all.toarray())

# 5. SMOTE 적용 (소수 클래스만 오버샘플링) - Apply SMOTE after imputation
print("\nBefore SMOTE:")
print(pd.Series(y_train_encoded).value_counts())

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_imputed, y_train_encoded) # Use imputed and encoded data

print("\nAfter SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# 6. 최종 데이터셋 크기 확인
print("\n--- Final Shapes for Model Training ---")
print("Shape of X_train_resampled:", X_train_resampled.shape)
print("Shape of X_test_imputed:", X_test_imputed.shape)
print("Shape of y_train_resampled:", y_train_resampled.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

Class mapping: {'부적합': np.int64(0), '적합': np.int64(1)}

Before SMOTE:
1    104075
0       450
Name: count, dtype: int64

After SMOTE:
1    104075
0    104075
Name: count, dtype: int64

--- Final Shapes for Model Training ---
Shape of X_train_resampled: (208150, 844)
Shape of X_test_imputed: (26132, 844)
Shape of y_train_resampled: (208150,)
Shape of y_test_encoded: (26132,)


# XGBoost Classifier 모델을 초기화하고, 클래스 불균형을 고려하여 `scale_pos_weight`를 설정합니다. 학습 데이터로 모델을 학습시키고, 테스트 데이터에 대한 예측을 수행한 후 다양한 분류 성능 지표(Accuracy, Precision, Recall, F1-score, Confusion Matrix, ROC-AUC)를 계산하고 출력합니다.

# XGBoost Classifier 학습 중 발생한 오류를 해결하기 위해, `X_train`과 `X_test` 데이터프레임에서 'object' 타입의 컬럼들을 'category' 타입으로 변환합니다. 변환 후 다시 분류 모델을 학습시키고 성능 지표를 출력합니다.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# -----------------------------
# 1. 타겟 변수 인코딩 (이미 SMOTE 적용 전 데이터 사용)
# -----------------------------
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train['JDGMNT_WORD_NAME'])
y_test_encoded = le.transform(y_test['JDGMNT_WORD_NAME'])

print("\nClass mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# -----------------------------
# 2. XGBoost 모델 초기화
# -----------------------------
# SMOTE 적용 후 이미 클래스 균형이 맞춰졌으므로 scale_pos_weight 제거
xgb_classifier = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,  # 최신 버전에서 deprecated warning 방지
    random_state=42
)

# -----------------------------
# 3. SMOTE 적용 후 데이터로 학습
# -----------------------------
print("\nRetraining XGBoost Classifier with SMOTE-resampled data...")
xgb_classifier.fit(X_train_resampled, y_train_resampled)
print("Training complete.")

# -----------------------------
# 4. 예측 및 평가
# -----------------------------
# X_test_imputed를 그대로 사용
y_pred_class = xgb_classifier.predict(X_test_imputed)
y_pred_proba = xgb_classifier.predict_proba(X_test_imputed)[:, 1]

accuracy = accuracy_score(y_test_encoded, y_pred_class)
precision = precision_score(y_test_encoded, y_pred_class)
recall = recall_score(y_test_encoded, y_pred_class)
f1 = f1_score(y_test_encoded, y_pred_class)

print(f"\nClassifier Performance Metrics (after SMOTE + XGBoost):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

conf_matrix = confusion_matrix(y_test_encoded, y_pred_class)
print("\nConfusion Matrix (after SMOTE + XGBoost):")
print(conf_matrix)

roc_auc = roc_auc_score(y_test_encoded, y_pred_proba)
print(f"\nROC-AUC Score (after SMOTE + XGBoost): {roc_auc:.4f}")



Class mapping: {'부적합': np.int64(0), '적합': np.int64(1)}

Retraining XGBoost Classifier with SMOTE-resampled data...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:46:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training complete.

Classifier Performance Metrics (after SMOTE + XGBoost):
Accuracy: 0.9720
Precision: 0.9976
Recall: 0.9742
F1-score: 0.9858

Confusion Matrix (after SMOTE + XGBoost):
[[   52    60]
 [  671 25349]]

ROC-AUC Score (after SMOTE + XGBoost): 0.8603
